# Your Name: Kiejstut Bunikiewicz

### Assignment Name: CA02 - Spam eMail Detection using Naive Bayes Classification Algorithm

#### This program is Spam Classifier for eMails that uses the Naive Bayes supervised machine learning algorithm.

# Program Initialization Section 

### Enter Your Import Packages Here

In [6]:
# import packages here
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from google.colab import drive

#Mounting Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data File Reading Section

Write Code to Read Data Sources Here

In [8]:
# although they will be used until after the functions are written, we will read the data here
# make sure to mount your drive first via the pop-out on the left, the third icon will allow you to mount your google drive
TRAINING_EMAILS = '/content/drive/My Drive/MSBA Colab 2020/ML Algorithms/CA02/Data/train-mails'
TESTING_EMAILS = '/content/drive/My Drive/MSBA Colab 2020/ML Algorithms/CA02/Data/test-mails'

### Defining the Dict_Maker function

This function builds a Dictionary of most common 3000 words from all the email content. First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.


In [9]:
def Dict_Maker(base_dir):
  complete_words_list = []
  email_list = [os.path.join(base_dir,file) for file in os.listdir(base_dir)] #creating a list of files from the email folder
  for email in email_list:
    with open(email) as e:
      for sentence in e:
        word = sentence.split() #splitting the emails into the individual words
        complete_words_list += word #appending the individual words to the list
  words_dict = Counter(complete_words_list) #counting the number of words in the list
  list_to_remove = list(words_dict) #transforming the count back into a list

  for mail in list_to_remove:
    if mail.isalpha() == False:
      del words_dict[mail] # this is deleting words that are not alphanumeric characters
    elif len(mail) == 1:
      del words_dict[mail] #this line is deleting words 1 letter long, AKA "stop words" like "a" 
  words_dict = words_dict.most_common(3000) #this is creating a dictionary of the 3000 most common words from the cleaned set of words
  return words_dict

### Defining the Mail_Word_Labeller Function

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). The function also analyzes the File Names of each email file and decides if it's a Spame or not based on the naming convention. Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.


In [10]:
def Mail_Word_Labeller(emailing_dir):
  mails = [os.path.join(emailing_dir,filed) for filed in os.listdir(emailing_dir)] #creating a list of files from the folder
  email_features_matrix = np.zeros((len(mails),3000)) #creating an empty matrix for number of files and 3000 most common words
  training_labels = np.zeros(len(mails)) #creating an empty set of labels by length of number of files
  wordcount = 1;
  emailID = 0;
  for parcel in mails: #looping through listed files
    with open(parcel) as post:
      for ind, line in enumerate(post): #enumerating the file that is being looped through
        if ind ==2:
          words = line.split() #splitting the files into individual words
          for word in words:
            wordID = 0 #setting a base wordID
            for ind, d in enumerate(words_dict):
              if d[0] == word:
                wordID = ind #these three lines are matching the word that is being looped through with the matching word in the dictionary
                email_features_matrix[emailID,wordID] = words.count(word) #if there is a match, the number of occurrences is counted
      training_labels[emailID] = 0; #this is setting a base emailID, assuming the email is not spam
      filepathTokens = parcel.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        training_labels[emailID] = 1; #the email id is reassigned if the end of the file name indicates the training data is a spam file
        wordcount = wordcount + 1 # if the file is spam, the wordcount variable is changed, indicating an occurence of the word in a spam file
      emailID = emailID + 1 #this file is creating a new emailID for the next email
  return email_features_matrix, training_labels


## Running the Naive Bayes, Sanity Check, and Analysis

The section is the main Program that calls the above two functions and gets executed first. First it "trains" the model using model.fit function and Training Dataset. After that it scores the Test Data set by running the Trained Model with the Test Data set. At the end it prints the model performance in terms of accuracy score.

In [11]:
# Finding Popular Words
words_dict = Dict_Maker(TRAINING_EMAILS) #creating a base dictionary of most common words using the training set

# Extracting the Counts and ID from the Test and Training Sets
print ("Reading and Processing emails from train-mails and test-mails folders")
email_features_matrix, training_labels = Mail_Word_Labeller(TRAINING_EMAILS) #this extracts ID (the emailid# and the wordID #), whether email is spam, and whether the popular words are within the training set
testing_features_matrix, test_labels = Mail_Word_Labeller(TESTING_EMAILS) #same, but for test set 

#Gauss Naive Bayes Model
model = GaussianNB()  #we are using a model that assumes the words are normally distributed and independent of each other

print ("Training Model using Gaussian Naive Bayes algorithm .....")
model.fit(email_features_matrix, training_labels) #fitting the model using the training set. this is creating a set of probability of a specific word occurring in spam vs. nonspam emails

#Checking features
print("Test of Whether Number of Occurrences are Being Recorded...")
print(list(email_features_matrix)[1]) #function is successfully counting the number of occurrences of the popular words in each email
print("Testing if Nonspam/Spam Distinciton is Being Made...")
print(list(training_labels)[0:10]) #function is outputting indications of whether the word is spam(1) or not(0)

#Sanity Check to See if Data was correctly transferred from the files
# we should have an equal number of spam and nonspam data in the training data, adding up to 702
print("Sanity Check by counting number of files")
print("Number of Nonspam in Training...")
print(list(training_labels).count(0))
print("Number of Spam in Training...")
print(list(training_labels).count(1)) #351 emails in each category, this matches our expectations

print ("Training completed")
print ("Testing trained model to predict Test eMail labels...")
predicted_labels = model.predict(testing_features_matrix) #predicting the spam v. nonspam with the test set
print(list(predicted_labels)[0:5]) #as seen from the list, the numbers are designating 0 for nonspam and 1 for spam

#count of predicted nonspam and spam vs. real count
print('Predicted Number of Nonspam in Test Data:')
print(list(predicted_labels).count(0)) #137 nonspam predicted
print('Predicted Number of Spam in Test Data:') 
print(list(predicted_labels).count(1)) #123 spam predicted
print('Actual Number of Nonspam in Test Data:')
print(list(test_labels).count(0)) #130 nonspam actual
print('Actual Number of Spam in Test Data:') 
print(list(test_labels).count(1)) #130 spam actual

# a small number of spam emails get through, but it does not look like there are any false positives
#This is because the number of spam is lower than expected. However, there could be both types of errors.

print ("Completed classification of the Test eMail Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels)) #comparing the correct answers (supervised learing) with the model's predictions(TRAINING_EMAILS)
# It looks like 96.5% is a good accuracy rate. I think the number of emails falsely described as spam would increase if we set the default to assume
# that the emails were spam.

Reading and Processing emails from train-mails and test-mails folders
Training Model using Gaussian Naive Bayes algorithm .....
Test of Whether Number of Occurrences are Being Recorded...
[3. 2. 3. ... 0. 0. 0.]
Testing if Nonspam/Spam Distinciton is Being Made...
[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Sanity Check by counting number of files
Number of Nonspam in Training...
351
Number of Spam in Training...
351
Training completed
Testing trained model to predict Test eMail labels...
[1.0, 1.0, 0.0, 0.0, 0.0]
Predicted Number of Nonspam in Test Data:
137
Predicted Number of Spam in Test Data:
123
Actual Number of Nonspam in Test Data:
130
Actual Number of Spam in Test Data:
130
Completed classification of the Test eMail Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
0.9653846153846154
